##
```python
class class_or_instancemethod(classmethod):
    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, owner)
```
```python
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.f), '__get__'):
            # 将类对象绑定为第一个参数
            return self.f.__get__(cls, type(cls))
        return MethodType(self.f, cls)
```

## `<class 'function'>` 和 `<class 'method'>`